## Classifying mangrove damage and recover due to tropical cyclones
- based on Emmas_code_TC_mangroves.py
- need to fix up for mangroves collection 3 (30m not 25m....)
- need to use static geotiffs in google drive for the windfield data input

### NCI data
/g/data/u46/users/ea6141/more_mangroves  

cyclone_damage_tally_notsure folder means it's not sure if mangrove is there on not (i.e. only 3 or 4 obs for the year)

/g/data/r78/mangroves



### workflow
- load in geotiff of windfield models
- select out area from geotiff for DEA mangroves load



https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Reprojecting_data.html


load in geotiff as dataset

load bc_datsets

cut out geotiff and use



In [53]:
import datacube
from datacube.model import Measurement
import numpy as np
import xarray as xr
from os import path
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.utils.geometry import CRS
import pandas as pd
import os
from  datacube import Datacube
from datacube.utils import geometry
import pickle
import sys
import rioxarray
dc = datacube.Datacube(app="cyclone mangroves")

from datacube.testutils.io import rio_slurp_xarray

In [72]:
windspeed_category = {'C1': [0., 125*1000/60**2],
                        'C2':[125*1000/60**2, 165*1000/60**2],
                        'C3': [165*1000/60**2, 225*1000/60**2], 
                        'C4': [225*1000/60**2, 280*1000/60**2],
                        'C5': [280*1000/60**2, 9999.]}

### load in static geotiff, and add time dim

In [54]:
geotiff_path = './Yasi_25m_EPSG3577.tif'
# Open into an xarray.DataArray
geotiff_da = rioxarray.open_rasterio(geotiff_path)
# Covert our xarray.DataArray into a xarray.Dataset
geotiff_ds = geotiff_da.to_dataset('band')
# Rename the variable to a more useful name
dataset = geotiff_ds.rename({1: 'wind_speed'})

### load mangroves based on before and after cyclone and extent of geotiff

In [57]:
# get extent of cyclone dataset
geobox = dataset.extent
# get the cyclone start time
start_time = 2011

# get time of cyclone
time_of_cyclone = (str(start_time) + '-01-01', str(start_time) + '-12-31')
# get time query before cyclone
time_before_cyclone = (str(start_time - 1) + '-01-01', str(start_time - 1) + '-12-31')
# get time query after cyclone
time_after_cyclone = (str(start_time) + '-01-01', '2021-12-31')

# find all datasets for mangrove canopy cover at cyclone time
time_of_cyclone_datasets = dc.find_datasets(product = 'ga_ls_mangrove_cover_cyear_3',time=time_of_cyclone, geopolygon=geobox)

# before cyclone mangrove canopy cover datasets, group by time
time_of_cyclone_datasets = dc.group_datasets(time_of_cyclone_datasets, group_by='time')

In [58]:
full_box = geometry.GeoBox.from_geopolygon(geobox, resolution=(25, -25))
# measurements for mangrove canopy cover
measurement = [Measurement(name='canopy_cover_class', dtype='int16', nodata=-1, units='1')]

time_of_cyclone_data = dc.load_data(time_of_cyclone_datasets, geobox = full_box, measurements=measurement)

In [59]:
cyclone_da = rio_slurp_xarray(geotiff_path, gbox=time_of_cyclone_data.geobox)
cyclone_da_time = cyclone_da.expand_dims(dim={"time": time_of_cyclone_data.time})
cyclone_ds = cyclone_da_time.to_dataset(name="wind_speed")

In [60]:
# copy over attributes
cyclone_ds.attrs['grid_mapping'] = time_of_cyclone_data.attrs['grid_mapping']
cyclone_ds.attrs['crs'] = time_of_cyclone_data.attrs['crs']

In [61]:
cyclone_ds

<xarray.Dataset>
Dimensions:      (time: 1, y: 12148, x: 7978)
Coordinates:
  * time         (time) datetime64[ns] 2011-07-02T11:59:59.999999
  * y            (y) float64 -2.197e+06 -2.197e+06 ... -1.893e+06 -1.893e+06
  * x            (x) float64 1.586e+06 1.586e+06 ... 1.387e+06 1.387e+06
    spatial_ref  int32 3577
Data variables:
    wind_speed   (time, y, x) float32 -9.999e+03 -9.999e+03 ... -9.999e+03
Attributes:
    grid_mapping:  spatial_ref
    crs:           PROJCS["GDA94 / Australian Albers",GEOGCS["GDA94",DATUM["G...

In [62]:
# find all datasets for mangrove canopy cover before cyclone
bc_datasets = dc.find_datasets(product = 'ga_ls_mangrove_cover_cyear_3',time=time_before_cyclone, geopolygon=geobox)

In [63]:
cyclone_extent = {}
k = 0
for d in bc_datasets:
    cyclone_extent[k] = d.extent
    k += 1

In [65]:
# just get first polygon for testing
first_value = list(cyclone_extent.values())[0]


In [66]:
# before cyclone mangrove canopy cover datasets, group by time
bc_datasets = dc.group_datasets(bc_datasets, group_by='time')
# find all datasets for mangrove canopy cover after cyclone
ac_datasets = dc.find_datasets(product = 'ga_ls_mangrove_cover_cyear_3',time=time_after_cyclone, geopolygon=geobox)
# after cyclone mangrove canopy cover datasets, group by time
ac_datasets = dc.group_datasets(ac_datasets, group_by='time')

In [67]:
# get polygon extent of cyclone # just using first value for testing
loading_polygon = first_value.intersection(geobox.to_crs(first_value.crs))
# get bounding box of polygon with 25m res
loading_box = geometry.GeoBox.from_geopolygon(loading_polygon, resolution=(25, -25))

### damage_level_by_geo function

In [75]:
def categorize_damage(bc_canopy, ac_canopy):
    '''
    On the mangrove classes. Difference before cyclone and after cyclone.
    '''
    reduction = bc_canopy - ac_canopy
    return reduction

In [73]:
'''
cyclone_datasets = cyclone_ds
bc_datasets = bc_datasets
ac_datasets = ac_datasets
loading_box = loading_box
windspeed_category = windspeed_category
'''

'\ncyclone_datasets = cyclone_ds\nbc_datasets = bc_datasets\nac_datasets = ac_datasets\nloading_box = loading_box\nwindspeed_category = windspeed_category\n'

In [76]:
# measurements for mangrove canopy cover
measurement = [Measurement(name='canopy_cover_class', dtype='int16', nodata=-1, units='1')]
# load mangrove canopy cover from find.datasets that are before the cyclone
bc_canopy = dc.load_data(bc_datasets, geobox = loading_box, measurements=measurement)

immediate = 0

In [93]:
dir_name = './'
cyclone_name = 'Yasi'
cyclone_datasets = cyclone_ds

In [94]:
def damage_level_by_geo(dir_name, cyclone_name, cyclone_datasets, bc_datasets, ac_datasets, loading_box, windspeed_category, dump=True):
    '''
    dc: datacube (was initial a dev cube that why it's specified in this function)
    dir_name: directory output for results
    cyclone_name: as above in dictionary
    cyclone_datasets: ###### not sure what this loads ######
    bc_datasets: mangrove canopy cover datasets before the cyclone (using time from cyclone dataset)
    ac_datasets: mangrove canopy cover datasets after the cyclone (using time from cyclone dataset)
    loading_box: bounding box (25m res) of the cyclone extent (originally from polygon?)
    windspeed_category: as above in dictionary
    dump: writing out cyclone damage results
    '''
    
    # load windfield data (was indexed into dev dc)
    cyclone_data = cyclone_datasets
    # measurements for mangrove canopy cover
    measurement = [Measurement(name='canopy_cover_class', dtype='int16', nodata=-1, units='1')]
    # load mangrove canopy cover from find.datasets that are before the cyclone
    bc_canopy = dc.load_data(bc_datasets, geobox = loading_box, measurements=measurement)
    
    immediate = 0
    
    # make sure dir exists, otherwise make, for result outputs
    dir_name += '/' + cyclone_name
    if path.exists(dir_name) == False:
        os.mkdir(dir_name)
    
    # for time in after cyclone mangrove canopy cover datasets
    for t in ac_datasets.time.data:
        # load the data for the time step (they are not loaded yet, just find.datasets has been used)
        ac_canopy = dc.load_data(ac_datasets.sel(time=[t]), geobox = loading_box, measurements=measurement)
        # use the categorize_damage function above to work out damage (not sure how this works with multiple times for bc_datasets)
        damage_label = categorize_damage(bc_canopy.canopy_cover_class.data, ac_canopy.canopy_cover_class.data)
        
        ### relabel the categorize_damage data
        # -1 no mangroves
        # 0 is not observed/missing data
        # Immediate damage is 0-4 for before minus after mangrove class
        damage_label[np.logical_and(bc_canopy.canopy_cover_class.data == -1, ac_canopy.canopy_cover_class.data == -1)] = -1 
        damage_label[np.logical_or(bc_canopy.canopy_cover_class.data == 0, ac_canopy.canopy_cover_class.data == 0)] = -1 
        # Open forest to no mangroves == cat 4 damage
        damage_label[np.logical_and(bc_canopy.canopy_cover_class.data == 2, ac_canopy.canopy_cover_class.data == -1)] = 4 # reduce from 2 
        # Closed forest to no mangroves == cat 4 damage
        damage_label[np.logical_and(bc_canopy.canopy_cover_class.data == 3, ac_canopy.canopy_cover_class.data == -1)] = 4 # reduce from 3
        # Open woodland to no mangroves == cat 3 damage
        damage_label[np.logical_and(bc_canopy.canopy_cover_class.data == 1, ac_canopy.canopy_cover_class.data == -1)] = 3 # reduce from 1 
        damage_label[damage_label < 0] = -1
        
        # writing out category immediate damage after cyclone data to xarray to export as netcdf
        ac_canopy.time.attrs['units'] = "seconds since 1970-01-01 00:00:00"
        result  = xr.Dataset({"damage_level":(['time', 'y', 'x'], damage_label.astype('int16'))},
                    coords={'time':ac_canopy.time, 'y': ac_canopy.y, 'x':ac_canopy.x},
                    attrs={'crs': CRS('EPSG:3577'), 'nodata': -1})
        fout = '_'.join([cyclone_name, "%s" % int(ac_canopy.x.data.min()/10000), 
            "%s" % int(ac_canopy.y.data.min()/10000), 
            "%.10s" % str(bc_canopy.time[0].data), 
            "%.10s" % str(ac_canopy.time[0].data)]) + '.nc'
        
        if dump:
            # writing out just the damage categories
            # immediate damage; before minus after
            write_dataset_to_netcdf(result, path.join(dir_name, fout))
        
        # now add in the wind speed and calculate immediate impact of the cyclone by area
        # 5 wind speed categories so a loop with five iterations
        if immediate == 0:
            wind_cat = pd.DataFrame(index=list(windspeed_category.keys()), columns=np.arange(0,5))
            for key, value in windspeed_category.items():
                wind_damage = result.damage_level[0].where(np.logical_and(cyclone_data.wind_speed[0] >= value[0] 
                                            ,cyclone_data.wind_speed[0] < value[1]))
                for i in range(0, 5): # damage levels
                    wind_cat[i][key] = wind_damage.where(wind_damage == i).count().data * 0.025**2
            print(wind_cat)
            all_damage_level = result.damage_level
        else:
            all_damage_level = xr.concat([all_damage_level, result.damage_level], dim='time')
        immediate += 1

    # categorise long term damage from the cyclone
    # categories 1-5 for long term damage using time
    results = np.zeros(all_damage_level.shape[1:], dtype='int16')
    tmp = all_damage_level.where(np.logical_and(all_damage_level<=2, all_damage_level > 0)).count(dim='time')
    results[tmp.values > 0] = 1 # at least once
    results[tmp.values==all_damage_level.time.shape[0]] = 2 # for whole time period after cyclone

    tmp = all_damage_level.where(np.logical_and(all_damage_level<=4, all_damage_level > 2)).count(dim='time')
    results[tmp.values > 0] = 3 # at least once
    results[tmp.values==all_damage_level.time.shape[0]] = 4 # for whole time after cyclone

    tmp = all_damage_level.where(all_damage_level==4).count(dim='time') # always a level 4 - never recovered
    results[tmp.values==all_damage_level.time.shape[0]] = 5
    results[all_damage_level.values[0]==-1] = -1

    
    # writing out category long term damage after cyclone data to xarray to export as netcdf
    all_damage_level.time.attrs['units'] = "seconds since 1970-01-01 00:00:00"
    results = results.reshape((1, ) + results.shape)
    results = xr.Dataset({"damage_level":(['time', 'y', 'x'], results.astype('int16'))},
                coords={'time':all_damage_level.time[-1:], 'y': all_damage_level.y, 'x': all_damage_level.x},
                attrs={'crs': CRS('EPSG:3577'), 'nodata': -1})
    fout = '_'.join([cyclone_name, "%s" % int(all_damage_level.x.data.min()/10000), 
        "%s" % int(all_damage_level.y.data.min()/10000)]) + '_all.nc'
    
    if dump:
        write_dataset_to_netcdf(results, path.join(dir_name, fout))
    all_wind_cat = pd.DataFrame(index=list(windspeed_category.keys()), columns=np.arange(1,6))
    
    for key, value in windspeed_category.items():
        wind_damage = results.damage_level[0].where(np.logical_and(cyclone_data.wind_speed[0] >= value[0],
                                cyclone_data.wind_speed[0] < value[1]))
        for i in range(0, 5):
            all_wind_cat[i+1][key] = wind_damage.where(wind_damage == (i+1)).count().data * 0.025**2
    print(all_wind_cat)

    return wind_cat, all_wind_cat

In [95]:
wind_cat, all_wind_cat = damage_level_by_geo(dir_name, cyclone_name, cyclone_datasets, bc_datasets, ac_datasets, loading_box, windspeed_category, dump=True)



           0       1         2         3         4
C1       0.0     0.0       0.0       0.0       0.0
C2       0.0     0.0       0.0       0.0       0.0
C3  6.063125  3.1025  0.135625  0.041875  0.005625
C4       0.0     0.0       0.0       0.0       0.0
C5       0.0     0.0       0.0       0.0       0.0
           1        2         3        4       5
C1       0.0      0.0       0.0      0.0     0.0
C2       0.0      0.0       0.0      0.0     0.0
C3  4.234375  0.44375  0.175625  0.00625  0.0025
C4       0.0      0.0       0.0      0.0     0.0
C5       0.0      0.0       0.0      0.0     0.0
